<a href="https://colab.research.google.com/github/MorenoLaQuatra/DeepNLP/blob/main/practices/P4/Practice_4_NER_and_Intent_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deep Natural Language Processing @ PoliTO**

---


**Teaching Assistant:** Moreno La Quatra

**Practice 3:** Named Entities Recognition & Intent Detection

## Named Entities Recognition (NER)

The Named Entity Recognition task aims at identifying and classifying named entities in a text. Named entities are real-world objects such as persons, locations, organizations, etc. The task takes as input a sentence and determines the boundaries of the named entities and their type. 

For example, given the sentence:

```
I went to Paris last week.
```

the task is to identify the named entity `Paris` as a location.

Hereafter an illustration of the NER task:

![https://miro.medium.com/max/875/0*mlwDqNm7DFc_4maP.jpeg](https://miro.medium.com/max/875/0*mlwDqNm7DFc_4maP.jpeg)   

In the first part of this practice, you will:
- explore the NER task using pre-trained models available on Spacy and HuggingFace
- evaluate the performance of a SpaCy NER model on a custom dataset
- evaluate the performance of a HuggingFace NER model on a custom dataset

NB: the metric used to evaluate the performance of the models is seqeval, which is a library for evaluating sequence labeling tasks.

### **Question 1: data preparation**

The first step is to prepare the data. In this practice, you will use the WikiGold dataset[1][2], which is a collection of annotated sentences from Wikipedia. The dataset is available in [CONLL](https://simpletransformers.ai/docs/ner-data-formats/#text-file-in-conll-format) format. The dataset is available [here](https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P4/NER/wikigold.conll.txt). 

**Please, read carefully the following instructions before starting to work on the practice.**

You need to extract clean sentences (no annotation) and, for each sentence, the corresponding annotations. The dataset has the following format:

- `sentences`: list of sentences
- `annotations`: list of list of entities (both string and class information). E.g., `[[('010', 'MISC'), ('Japanese', 'MISC'), ('The Mad Capsule Markets', 'ORG')], [('Osc-Dis', 'MISC'), ('Introduction 010', 'MISC'), ('Come', 'MISC')], ...]`. You can remove I- prefix because the data collection does not actually cotains valuable prefixes.

---


[1] Balasuriya, Dominic, et al. "Named entity recognition in wikipedia."
    Proceedings of the 2009 Workshop on The People's Web Meets NLP: Collaboratively Constructed Semantic Resources. Association for Computational Linguistics, 2009.

[2] Nothman, Joel, et al. "Learning multilingual named entity recognition
    from Wikipedia." Artificial Intelligence 194 (2013): 151-175 

---

The following cell downloads the dataset on Google Colab.

In [ ]:
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P4/NER/wikigold.conll.txt

To avoid spending too much time on data processing, the following cell prepare the dataset for you.
After running the cell, you will have the following variables:
- `sentences_with_labels`: a list of tokens with their corresponding labels
- `sentences`: a list of the sentences in the dataset
- `labels`: a list of lists of labels. Each element in the outer list corresponds to a list of labels for a sentence in the dataset.

Hereafter an example of the provided data:

```python
sentences_with_labels[0] = [
    ['010', 'I-MISC'], 
    ['is', 'O'], 
    ['the', 'O'], 
    ['tenth', 'O'], 
    ['album', 'O'], 
    ['from', 'O'], 
    ['Japanese', 'I-MISC'], 
    ['Punk', 'O'], 
    ['Techno', 'O'], 
    ['band', 'O'], 
    ['The', 'I-ORG'], 
    ['Mad', 'I-ORG'], 
    ['Capsule', 'I-ORG'], 
    ['Markets', 'I-ORG'], 
    ['.', 'O']
]

sentences[0] = [
    '010 is the tenth album from Japanese Punk Techno band The Mad Capsule Markets .'
]

labels[0] = [
    ('010', 'MISC'), 
    ('Japanese', 'MISC'), 
    ('The Mad Capsule Markets', 'ORG')
]
```

Please, note that the labels are not in IOB format. You can ignore the I- prefix because the data collection does not actually contains valuable prefixes.
Get familar with the data by printing the first 10 sentences and their corresponding labels. Which are the labels in the dataset?

In [ ]:
%%capture
! pip install datasets
! pip install transformers
! pip install spacy
! python -m spacy download en_core_web_sm

In [ ]:
def split_text_label(filename):
    f = open(filename)
    split_labeled_text = []
    sentence = []
    for line in f:
        if len(line)==0 or line.startswith('-DOCSTART') or line[0]=="\n":
             if len(sentence) > 0:
                 split_labeled_text.append(sentence)
                 sentence = []
             continue
        splits = line.split(' ')
        sentence.append([splits[0],splits[-1].rstrip("\n")])
    if len(sentence) > 0:
        split_labeled_text.append(sentence)
        sentence = []
    return split_labeled_text
sentences_with_labels = split_text_label("wikigold.conll.txt")

In [ ]:
print (sentences_with_labels[0])

sentences = [] 

for sent_list in sentences_with_labels:
    sentence = [s[0] for s in sent_list] 
    sentence = " ".join(sentence)
    sentences.append(sentence)

In [ ]:
print (sentences[0])

In [ ]:
labels = []
overall_labels = [] 
for sent_list in sentences_with_labels:
    current_labels = []
    prev = "O"
    current_entity = ""
    for w, l in sent_list:
        overall_labels.append(l)
        if l != "O" and prev != "O":
            if l == prev:
                # continue entity
                current_entity += w + " "
            else:
                # end prev and start a new one
                current_labels.append((current_entity.strip(), prev.split("-")[1]))
                current_entity = w + " "
        elif l == "O" and  prev != "O": 
            # end prev
            current_labels.append((current_entity.strip(), prev.split("-")[1]))
            current_entity = ""
        elif l != "O" and prev == "O":
            # start new
            current_entity = w + " "

        prev = l
    labels.append(current_labels)

print (labels)
overall_labels = list(set(overall_labels))
overall_labels = [o for o in overall_labels if o != "O"] 
overall_labels = [o.split("-")[1] for o in overall_labels] 
print (overall_labels)

In [ ]:
print (labels[0])

In [ ]:
# Your code and answers here

### **Question 2: inference with spacy for entity recognition**

SpaCy is a free, open-source library for advanced Natural Language Processing in Python. It features NER models for different languages including English.
The models are available [here](https://spacy.io/models). 

For this question you asked to instantiate a spacy model for English and perform inference on the sentences in the dataset. The English model contains a superset of the labels in the dataset. For this reason, you need to map the labels that are not in the dataset to the `MISC` label.

You are expected to generate an output similar to the following:
```python
[('010', 'MISC'), ('Japanese', 'MISC'), ('The Mad Capsule Markets', 'ORG')]
```

Please pay attention to the token attributes (you can find more information [here](https://spacy.io/api/token#attributes)) and the entity attributes (you can find more information [here](https://spacy.io/api/entityrecognizer)).

The following cell instantiates a spacy model for English.

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Your code here

### **Question 3: compute metrics for evaluating NER**

The output of NER models consists of a set of named entities. To evaluate the performance of a model, we need to compare the predicted named entities with the ground truth.

For this question, you need to use [`eval4ner`](https://github.com/cyk1337/eval4ner) package to evaluate the performance of the model. 

**Note**: please use `pip install git+https://github.com/MorenoLaQuatra/eval4ner` to use a fixed version of the library. Before passing the parameter to the evaluation function, create a deepcopy of each variable:
```python
from copy import deepcopy
sentences_copy = deepcopy(sentences) #pass sentences_copy to the eval script
```

The issue has been already reported to the original author.

In [ ]:
%%capture
! pip install git+https://github.com/MorenoLaQuatra/eval4ner

In [ ]:
# Your code here

### **Question 4: inference with transformers pipeline**

Transformer-based models can be fine-tuned for token-level classification. The task is to classify each token in a sentence and assign it to a class.
The NER task is a token-level classification task and the models can be used for performing inference on the sentences in the dataset.

You can use the pipeline available on the HuggingFace [transformers library](https://huggingface.co/docs/transformers/main_classes/pipelines). The pipeline allows to perform inference on a list of sentences.

Evaluate the **standard** model using the pipeline (`pipe = pipeline("ner")`). Check the documentation here: https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TokenClassificationPipeline


A few notes about the question (**read carefully**):
1. The output of the pipeline differs with respect to spacy. Please be sure to process data correctly before running evaluation.
2. `ignore_labels` parameter could be used to exclude labels from the prediction.
3. `##` symbol is used when a token is a continuation of a previous one (Poli + ##TO). You may need to check this specific case to merge the tokens correctly.
4. Use seqeval to evaluate the performance of the model.

In [ ]:
%%capture
! pip install datasets transformers

In [ ]:
# Your code here

## Intent Detection

In data mining, intention mining or intent mining is the problem of determining a user's intention from logs of his/her behavior in interaction with a computer system, such as in search engines. Intent Detection is the identification and categorization of what a user online intended or wanted to find when they type or speak with a conversational agent (or a search engine).

![https://d33wubrfki0l68.cloudfront.net/32e2326762c75a0357ab1ae1976a60d4bbce724b/f4ac0/static/a5878ba6b0e4e77163dc07d07ecf2291/2b6c7/intent-classification-normal.png](https://d33wubrfki0l68.cloudfront.net/32e2326762c75a0357ab1ae1976a60d4bbce724b/f4ac0/static/a5878ba6b0e4e77163dc07d07ecf2291/2b6c7/intent-classification-normal.png)

In this section, you will use the [ATIS dataset](https://github.com/yvchen/JointSLU): https://github.com/yvchen/JointSLU ; https://www.kaggle.com/siddhadev/atis-dataset-clean/home

The task is to classify the intent of a sentence. The dataset is split into train, validation and test sets. **Use the provided splits** to train and evaluate your models.


In [ ]:
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P4/IntentDetection/atis.train.csv
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P4/IntentDetection/atis.dev.csv
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P4/IntentDetection/atis.test.csv

### **Question 5: two-step classification model**

Train a classification model to identify the intent from a given sentence. The model is required to leverage on pretrained BERT model to generate sentence embeddings (important: **no fine-tuning**). The model is required to use the embeddings to perform classification.

Once extracted the embeddings, you can use any classifier you want. For example, you can use a linear classifier (e.g. Logistic Regression) or a neural network (e.g. MLP). For convenience, you can use the `sklearn` library for training the classifier (https://scikit-learn.org/stable/supervised_learning.html).

![https://github.com/MorenoLaQuatra/DeepNLP/blob/main/practices/P4/IntentDetection/no_finetuning.png?raw=true](https://github.com/MorenoLaQuatra/DeepNLP/blob/main/practices/P4/IntentDetection/no_finetuning.png?raw=true)


Assess the performance of the trained model (the model on top of BERT) on the test set by using the **classification accuracy**, **precision**, **recall** and **F1-score**. You can use the `sklearn` library for computing the metrics (https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).


Note: you can use the `sentence-transformers` library to generate sentence embeddings (https://www.sbert.net/docs/pretrained_models.html).

In [ ]:
# Your code here

### **Question 6: finetuning end-to-end classification model**

Another approach is to fine-tune the BERT model for the classification task. A classification head is added on top of the pretrained BERT model. The classification head is trained end-to-end with the BERT model.
This approach is more effective than the previous one because the model is trained end-to-end. However, the model requires more training time and resources.

Train a new BERT model for the task of [sequence classification](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification) (include BERT fine-tuning).  

![https://github.com/MorenoLaQuatra/DeepNLP/blob/main/practices/P4/IntentDetection/finetuning.png?raw=true](https://github.com/MorenoLaQuatra/DeepNLP/blob/main/practices/P4/IntentDetection/finetuning.png?raw=true)

Assess the performance of the generated model by using the same metrics used in the previous question.

Which model has better performance? Why?

In [ ]:
# Your code here